# Money data
- per month, days i went over/under budget -- bar chart with the allowance as a line?
- cash vs card?
- is there a pattern we can find with my spending? weekdays vs. weekends? a particular time of the month?
- category breakdown per month (and overall) -- pie chart
    - thing i spent the most money on in a category
    - category spending over time
- drill into specific things -- how much did i spend on [water, dried meat, etc.]. maybe pick one per country?
- is there something that jumps out as a way i could save money while travelling?


- data I need to get -- months that are missing, anything spent outside of the app, flights internationally, insurance

#### imports

In [2]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly.express as px
import plotly.graph_objects as go

import json

#### variables

In [3]:
regions = {
    'South America': ['Colombia','Ecuador','Peru','Bolivia','Brasil'],
    'Central America': ['Guatemala','Honduras','Nicaragua','El Salvador'],
    'Southern Africa': ['South Africa','Malawi','Mozambique','Eswatini','Lesotho']
}

In [31]:
countries = {
    'Colombia' : {
        'color': '#EFCA08'
    },
    'Ecuador' : {
        'color': '#FF9000'
    },
    'Peru' : {
        'color': '#DB5461'
    },
    'Bolivia' : {
        'color': '#A4243B'
    },
    'Brasil' : {
        'color': '#94BFA7'
    },
    'Guatemala' : {
        'color': '#86A5D9'
    },
    'Honduras' : {
        'color': '#0B3954'
    },
    'Nicaragua' : {
        'color': '#57C4E5'
    },
    'El Salvador': {
        'color': '#273043'
    },
    'South Africa': {
        'color': '#D4E09B'
    },
    'Malawi': {
        'color': '#D72638'
    },
    'Mozambique': {
        'color': '#2D5C1E'
    },
    'Eswatini': {
        'color': '#AFA3B8'
    },
    'Lesotho': {
        'color': '#8EF9F3'
    }
}

#### getting the data

In [30]:
timeline = pd.read_csv('../data/country_timeline.csv')
timeline

,country,entry_date,exit_date,num_of_days
0,Colombia,2024-03-06,2024-04-18,43
1,Ecuador,2024-04-18,2024-05-13,25
2,Peru,2024-05-13,2024-07-02,50
3,Bolivia,2024-07-02,2024-08-15,44
4,Brasil,2024-08-15,2024-12-19,126
5,Ecuador,2025-01-17,2025-02-15,29
6,Guatemala,2025-02-15,2025-03-08,21
7,Honduras,2025-03-08,2025-03-17,9
8,Nicaragua,2025-03-17,2025-04-01,15
9,El Salvador,2025-04-01,2025-04-16,15


In [8]:
df = pd.read_csv('../data/money_manager.csv')
df.columns = df.columns.str.lower()
df.head()

,date,account,category,subcategory,note,cad,income/expense,description,amount,currency,account.1
0,09/25/2025 0:08:37,Card,Other,NaN,Sending Rochelle's batteries,6.69,Expense,NaN,90.0,ZAR,6.69
1,09/25/2025 0:08:27,Card,🍜 Food,NaN,Panini,2.60,Expense,NaN,35.0,ZAR,2.60
2,09/24/2025 15:00:00,Card,🤑 Allowance,NaN,NaN,60.00,Income,NaN,60.0,CAD,60.00
3,09/23/2025 21:34:49,Cash,🎁 Gift,NaN,Handful of beaded animals,19.32,Expense,NaN,260.0,ZAR,19.32
4,09/23/2025 15:00:00,Card,🤑 Allowance,NaN,NaN,60.00,Income,NaN,60.0,CAD,60.00


In [35]:
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y %H:%M:%S')
df['simplified_date'] = df['date'].dt.strftime("%m/%d/%Y")
df['month'] = df['date'].dt.strftime("%B %Y")
df['year'] = df['date'].dt.strftime("%Y")
# df['month'] = pd.to_datetime(df['month'], format="%B %Y")
df.head()

,date,account,category,subcategory,note,cad,income/expense,description,amount,currency,account.1,month,year,month_num,simplified_date
0,2025-09-25 00:08:37,Card,Other,NaN,Sending Rochelle's batteries,6.69,Expense,NaN,90.0,ZAR,6.69,September 2025,2025,2025-09-01,09/25/2025
1,2025-09-25 00:08:27,Card,🍜 Food,NaN,Panini,2.60,Expense,NaN,35.0,ZAR,2.60,September 2025,2025,2025-09-01,09/25/2025
2,2025-09-24 15:00:00,Card,🤑 Allowance,NaN,NaN,60.00,Income,NaN,60.0,CAD,60.00,September 2025,2025,2025-09-01,09/24/2025
3,2025-09-23 21:34:49,Cash,🎁 Gift,NaN,Handful of beaded animals,19.32,Expense,NaN,260.0,ZAR,19.32,September 2025,2025,2025-09-01,09/23/2025
4,2025-09-23 15:00:00,Card,🤑 Allowance,NaN,NaN,60.00,Income,NaN,60.0,CAD,60.00,September 2025,2025,2025-09-01,09/23/2025


In [50]:
# we have a weird june 2024 data point
df = df[df.month != 'June 2024']

# and we have a duplicate data entry -- delete it
df = df.drop([1475], axis=0)

In [37]:
print ('spent money in {} different currencies'.format(df.currency.unique().shape[0]))
df.currency.unique()

spent money in 10 different currencies


array(['ZAR', 'CAD', 'MZN', 'USD', 'NIO', 'HNL', 'GTQ', 'BRL', 'PEN',
       'BOB'], dtype=object)

In [51]:
df[(df.category == '🤑 Allowance') & (df.simplified_date == '01/17/2025')]

,date,account,category,subcategory,note,cad,income/expense,description,amount,currency,account.1,month,year,month_num,simplified_date
1476,2025-01-17 22:00:00,Card,🤑 Allowance,NaN,NaN,60.0,Income,NaN,60.0,CAD,60.0,January 2025,2025,2025-01-01,01/17/2025
1478,2025-01-17 21:00:00,Card,🤑 Allowance,NaN,Additional Galapagos budget,40.0,Income,NaN,40.0,CAD,40.0,January 2025,2025,2025-01-01,01/17/2025


## Totals

In [63]:
allowance = df[df.category == '🤑 Allowance'].groupby('simplified_date').cad.sum().reset_index()
expense = df[df.category != '🤑 Allowance'].groupby('simplified_date').cad.sum().reset_index()

In [65]:
print ('Overall, for 15 months of travel')
print ('I budgeted ${}'.format(np.round(df[df['income/expense'] == 'Income'].cad.sum(),2)))
print ('I spent ${}'.format(np.round(df[df['income/expense'] == 'Expense'].cad.sum(),2)))
print ('Mean daily budget: ${}'.format(np.round(allowance.cad.mean(),2)))
print ('Mean daily spending: ${}'.format(np.round(expense.cad.mean(),2)))

Overall, for 15 months of travel
I budgeted $30250.0
I spent $27929.63
Mean daily budget: $71.68
Mean daily spending: $68.79


## Monthly Breakdown

In [22]:
df.groupby(['simplified_date','month','income/expense']).cad.sum().reset_index().sort_values('simplified_date', ascending=True)

,simplified_date,month,income/expense,cad
0,2024-07-01,July 2024,Expense,1811.48
1,2024-07-01,July 2024,Income,1860.00
2,2024-08-01,August 2024,Expense,2257.19
3,2024-08-01,August 2024,Income,2220.00
4,2024-09-01,September 2024,Expense,2022.80
5,2024-09-01,September 2024,Income,2400.00
6,2024-10-01,October 2024,Expense,2068.33
7,2024-10-01,October 2024,Income,2480.00
8,2024-11-01,November 2024,Expense,1906.31
9,2024-11-01,November 2024,Income,2400.00


make a line chart here to show expense vs. budget. color by country as well

In [ ]:
# need to group by because sometimes take different transport on one day
t_grp = transport.groupby(['country','date','approximate_time']).sum()

# creating the base line chart
fig = px.line(
    transport, 
    x='date', 
    y='approximate_time', 
    labels={
         'date': 'Date',
         'approximate_time': 'Hours'
    },
    title='Hours spent in transportation over time'
)

fig.update_traces(line_color='black')

y_max = transport.approximate_time.max()

# adding the different colours to represent different countries
for c in list(itertools.chain.from_iterable(regions.values())):
    clip = transport[transport.country == c]
    # having an issue with the boxes not matching up because there is a day between travels. need to get date from idx before
    # deal with this later

    # Ecuador case! two different parts of the trip
    if c == 'Ecuador':
        # first leg
        x_range = [clip[clip.region != 'Galapagos'].date.min(), clip[clip.region != 'Galapagos'].date.max()]
        add_rect(c, x_range, y_max)

        # second leg
        x_range = [clip[clip.region == 'Galapagos'].date.min(), clip[clip.region == 'Galapagos'].date.max()]
        add_rect(c, x_range, y_max)

    else: 
        x_range = [clip.date.min(), clip.date.max()]
        add_rect(c, x_range, y_max)

# fig.update_xaxes(tickformat="%B %Y")
fig.show()
